Topic Modeling

In [1]:
import pandas as pd
import numpy as np
import pickle


In [2]:
#Import Data
data = pd.read_csv('delhi-pollution-October-December-2019.csv', error_bad_lines=False, delimiter='    ')
#document = data['text']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [3]:
textData = data['Tweet text']

In [20]:
#x = textData[0:100]


In [4]:
# Apply a first round of text cleaning techniques
import re
import string
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text)
    text = re.sub('â€™', '', text)
    text = re.sub('œ', '', text)
    re.sub(r'[^\x00-\x7F]+','', text)
    return text

def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

#not doing stemming.
def lemmatize_stemming(text):
    #stemmer = SnowballStemmer("english")
    return WordNetLemmatizer().lemmatize(text, pos='v')

def remove_nonascii(text):
    return ''.join([i if ord(i) < 128 else ' ' for i in text])

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return " ".join(str(x) for x in result)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
textData = [clean_text_round1(item) for item in textData]
textData = [clean_text_round2(item) for item in textData]
textData = [preprocess(item) for item in textData]
#remove anything that is non-ascii
textData = [remove_nonascii(item) for item in textData]



textData = pd.DataFrame(textData)
textData.to_pickle("clean_data.pkl")

In [6]:
tokenLists = []
for item in textData[0]:
    tokenLists.append(item.split(' '))

In [7]:
dictionary = gensim.corpora.Dictionary(tokenLists)

In [8]:
dictionary.filter_extremes(no_below=15, no_above=0.5)

In [9]:
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenLists]

In [10]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [14]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=50, id2word=dictionary, passes=200, workers=2)

In [15]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.123*"chamber" + 0.115*"situation" + 0.114*"better" + 0.099*"turn" + 0.045*"alarm" + 0.036*"airpollution" + 0.032*"airport" + 0.031*"deaths" + 0.024*"send" + 0.021*"love"
Topic: 1 
Words: 0.123*"people" + 0.095*"work" + 0.077*"blame" + 0.059*"suffer" + 0.049*"free" + 0.031*"political" + 0.030*"party" + 0.026*"water" + 0.024*"instead" + 0.024*"game"
Topic: 2 
Words: 0.153*"smog" + 0.095*"toxic" + 0.050*"delhis" + 0.046*"see" + 0.040*"get" + 0.039*"flight" + 0.032*"bangladesh" + 0.029*"claim" + 0.025*"ahead" + 0.024*"didnt"
Topic: 3 
Words: 0.149*"capital" + 0.077*"national" + 0.074*"indian" + 0.056*"indias" + 0.054*"make" + 0.045*"people" + 0.031*"protest" + 0.029*"face" + 0.025*"region" + 0.025*"cities"
Topic: 4 
Words: 0.210*"quality" + 0.097*"hazardous" + 0.082*"airpollution" + 0.060*"index" + 0.054*"record" + 0.053*"noida" + 0.053*"level" + 0.029*"gurgaon" + 0.026*"chennai" + 0.026*"cross"
Topic: 5 
Words: 0.118*"share" + 0.082*"lung" + 0.072*"cancer" + 0.052*"stag

In [16]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [17]:
pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
38    -0.030796 -0.274962       1        1  3.781112
43     0.002863  0.005543       2        1  3.439874
37     0.284995  0.044092       3        1  3.385058
48     0.004995 -0.072837       4        1  2.873313
10    -0.076916 -0.128536       5        1  2.830790
40    -0.153822 -0.070987       6        1  2.806658
19     0.164227  0.108827       7        1  2.639793
47    -0.183531  0.082599       8        1  2.639173
27     0.049161  0.061884       9        1  2.553854
1     -0.177567  0.065580      10        1  2.510408
21     0.023663 -0.211144      11        1  2.486063
12    -0.167900  0.092831      12        1  2.435481
3     -0.100942 -0.091677      13        1  2.380172
4      0.147379 -0.155058      14        1  2.249194
34     0.128512 -0.021764      15        1  2.243063
29    -0.072535 -0.011954      16        1  2.230817
32    -0.042817 -0.136401      17        1  2.206597
8      0.112375 -0.094892      18        1  2.163991
36    -0.089573  0.079045      19        1  2.125227
13     0.004443  0.043235      20        1  2.107794
14     0.126804 -0.081504      21        1  2.083745
44    -0.025490 -0.005114      22        1  2.043972
2     -0.054566  0.062576      23        1  2.036644
39     0.141254 -0.073610      24        1  1.998948
42    -0.088558  0.041096      25        1  1.989844
11     0.016611  0.036837      26        1  1.935719
46    -0.082285  0.030859      27        1  1.903983
18     0.178126  0.014136      28        1  1.802383
28    -0.050592  0.009486      29        1  1.690547
22     0.010184  0.020447      30        1  1.671875
49     0.045609  0.089123      31        1  1.657604
30    -0.016756 -0.060299      32        1  1.642513
6      0.056397  0.051936      33        1  1.624739
17     0.131790  0.010036      34        1  1.616753
41     0.002397  0.006754      35        1  1.569501
35    -0.105014  0.118728      36        1  1.559230
24    -0.045023 -0.192961      37        1  1.541572
20    -0.135988  0.035450      38        1  1.530419
33    -0.096310  0.048256      39        1  1.487550
26     0.088611  0.040503      40        1  1.468106
16    -0.108301  0.006011      41        1  1.467328
31    -0.073696  0.069379      42        1  1.443426
23    -0.002053  0.063985      43        1  1.424155
15     0.084066  0.014762      44        1  1.423898
45     0.199781  0.129831      45        1  1.385057
0      0.019163  0.038431      46        1  1.248129
25     0.081230  0.178411      47        1  1.227699
7     -0.122565 -0.028373      48        1  1.185125
9     -0.005026  0.017880      49        1  1.180568
5      0.003986 -0.006475      50        1  1.070533, topic_info=    Category         Freq             Term        Total  loglift  logprob
0    Default  6241.000000                   6241.000000  30.0000  30.0000
47   Default  2434.000000            india  2434.000000  29.0000  29.0000
25   Default  1522.000000   arvindkejriwal  1522.000000  28.0000  28.0000
91   Default  1803.000000   delhipollution  1803.000000  27.0000  27.0000
84   Default  1197.000000             burn  1197.000000  26.0000  26.0000
..       ...          ...              ...          ...      ...      ...
83   Topic50    28.170902  airqualityindia    50.030109   3.9627  -4.2523
36   Topic50    53.702564             time   993.059082   1.6197  -3.6072
716  Topic50    29.350685            thank   190.514435   2.6666  -4.2113
311  Topic50    26.269039           impact   153.343262   2.7727  -4.3222
2    Topic50    40.136501     airpollution  7041.284668  -0.6302  -3.8983

[2111 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
0         2  0.999892               
1148     14  0.642566         aajtak
1148     15  0.355102         aajtak
1        28  0.996067  aamaadmiparty
371      12  0.924737      aapatwork
...     ...     

## LDA on Noun

In [143]:
tokenListsNounAdj = []

In [142]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return all_nouns

# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return nouns_adj

In [144]:
for item in tokenLists:
    tokenListsNounAdj.append(nouns_adj(' '.join(item)))

In [145]:
dictionary = gensim.corpora.Dictionary(tokenListsNounAdj)
dictionary.filter_extremes(no_below=15, no_above=0.5)
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenListsNounAdj]
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [146]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=100, id2word=dictionary, passes=100, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


Topic: 0 
Words: 0.267*"worst" + 0.104*"flight" + 0.071*"year" + 0.064*"vote" + 0.059*"better" + 0.056*"sure" + 0.050*"hard" + 0.047*"implement" + 0.031*"level" + 0.031*"annual"
Topic: 1 
Words: 0.432*"airpollution" + 0.150*"place" + 0.110*"mumbai" + 0.055*"divert" + 0.047*"airport" + 0.041*"ration" + 0.037*"terrible" + 0.027*"hazard" + 0.023*"heart" + 0.022*"youre"
Topic: 2 
Words: 0.089*"discuss" + 0.074*"picture" + 0.074*"airpollution" + 0.070*"debate" + 0.068*"parliament" + 0.055*"panel" + 0.055*"discussion" + 0.054*"october" + 0.049*"large" + 0.047*"sabha"
Topic: 3 
Words: 0.238*"increase" + 0.127*"come" + 0.096*"wind" + 0.068*"doctor" + 0.068*"entire" + 0.048*"speed" + 0.048*"likely" + 0.042*"level" + 0.039*"tuesday" + 0.039*"safar"
Topic: 4 
Words: 0.363*"burn" + 0.274*"stubble" + 0.168*"cause" + 0.069*"major" + 0.024*"delhis" + 0.017*"nasa" + 0.017*"warn" + 0.016*"shock" + 0.011*"image" + 0.010*"contributor"
Topic: 5 
Words: 0.145*"problems" + 0.107*"check" + 0.087*"healthy" + 

In [147]:
pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
73    -0.285640 -0.088903       1        1  2.367356
94     0.185912 -0.254457       2        1  2.180752
85     0.026302 -0.072276       3        1  1.988573
24    -0.161183 -0.040772       4        1  1.974895
31    -0.041241 -0.043428       5        1  1.720263
...         ...       ...     ...      ...       ...
84     0.058214  0.042856      96        1  0.499727
11     0.064068  0.029965      97        1  0.498505
67     0.091551  0.002414      98        1  0.473109
21     0.104713  0.041099      99        1  0.459871
99     0.012657  0.062557     100        1  0.446974

[100 rows x 5 columns], topic_info=     Category         Freq            Term        Total  loglift  logprob
59    Default  2122.000000           india  2122.000000  30.0000   30.000
76    Default  1784.000000  delhipollution  1784.000000  29.0000   29.000
19    Default  1282.000000  arvindkejriwal  1282.000000  28.0000   28.000
114   Default   930.000000            govt   930.000000  27.0000   27.000
17    Default  1236.000000         quality  1236.000000  26.0000   26.000
..        ...          ...             ...          ...      ...      ...
519  Topic100     0.011303          sudden    20.619381  -2.0985  -10.966
530  Topic100     0.011303          centre   215.369400  -4.4447  -10.966
520  Topic100     0.011303           break    94.493011  -3.6208  -10.966
528  Topic100     0.011303           limit   174.360916  -4.2334  -10.966
521  Topic100     0.011303            dead    16.485102  -1.8748  -10.966

[3897 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
974      27  0.789231         aajtak
974      71  0.192495         aajtak
0        23  0.996669  aamaadmiparty
305      86  0.932313      aapatwork
124      23  0.972999       aapdelhi
...     ...       ...            ...
1028     58  0.943114          youth
283      29  0.989480        youtube
513      75  0.935410   zeehindustan
1438     56  0.933861        zeenews
186      72  0.981539           zone

[2239 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[74, 95, 86, 25, 32, 23, 5, 83, 90, 93, 97, 99, 60, 52, 72, 17, 13, 65, 61, 76, 89, 31, 21, 43, 82, 59, 64, 36, 92, 26, 14, 78, 30, 94, 11, 39, 88, 35, 1, 19, 53, 38, 70, 75, 34, 84, 20, 49, 9, 98, 44, 33, 79, 73, 47, 24, 46, 37, 50, 40, 41, 66, 2, 63, 4, 45, 81, 7, 51, 58, 71, 28, 16, 3, 6, 69, 55, 15, 42, 67, 62, 10, 80, 56, 57, 27, 29, 48, 91, 8, 96, 77, 54, 87, 18, 85, 12, 68, 22, 100])

In [89]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rishiraj\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [91]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Rishiraj\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

### LDA using NMF

In [107]:
import re, nltk, spacy, string

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint


In [113]:
#Import Data
dataAP = pd.read_csv('ap-nov19.csv', error_bad_lines=False, delimiter='    ')
dataTrump = pd.read_csv('trump-nov19.csv', error_bad_lines=False, delimiter='    ')
#document = data['text']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [114]:
dataAP = dataAP['Tweet text']
dataTrump = dataTrump['Tweet text']

In [115]:
dataAP = [clean_text_round1(item) for item in dataAP]
dataAP = [clean_text_round2(item) for item in dataAP]
dataAP = [preprocess(item) for item in dataAP]


dataTrump = [clean_text_round1(item) for item in dataTrump]
dataTrump = [clean_text_round2(item) for item in dataTrump]
dataTrump = [preprocess(item) for item in dataTrump]


In [123]:
from sklearn.decomposition import NMF
n_features = 4000
n_components = 20
n_top_words = 20

# ignore terms that have a document frequency strictly higher than 95%, 
# ignore terms that have a document frequency strictly lower than 2
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(dataTrump)

# alpha=0 means no regularization, l1_ratio=.5, the penalty is a combination of L1 and L2
nmf = NMF(n_components=n_components, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)
nmf_output = nmf.fit_transform(tfidf)

def show_topics(vectorizer=tfidf_vectorizer, lda_model=nmf, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=tfidf_vectorizer, lda_model=nmf, n_words=20)        


# Trump Topics

In [124]:
topic_keywords

[array(['trump', 'time', 'elect', 'isnt', 'lie', 'need', 'work', 'hate',
        'talk', 'countri', 'real', 'year', 'sell', 'youtub', 'help', 'hat',
        'corrupt', 'administr', 'reason', 'cult'], dtype='<U15'),
 array(['realdonaldtrump', 'potus', 'donaldjtrumpjr', 'secpompeo',
        'return', 'whitehous', 'believ', 'marcorubio', 'victori', 'care',
        'bless', 'good', 'tuckercarlson', 'investig', 'view', 'christma',
        'tell', 'lie', 'senategop', 'ingrahamangl'], dtype='<U15'),
 array(['presid', 'bless', 'lincoln', 'best', 'greatest', 'militari',
        'youtub', 'countri', 'better', 'american', 'troop', 'obama',
        'ghani', 'happi', 'trump', 'serv', 'hold', 'time', 'year', 'coup'],
       dtype='<U15'),
 array(['know', 'better', 'thing', 'potus', 'didnt', 'need', 'wrong',
        'time', 'lincoln', 'sure', 'obama', 'shes', 'lie', 'doesnt',
        'brain', 'isnt', 'refus', 'whistleblow', 'wont', 'terrorist'],
       dtype='<U15'),
 array(['love', 'troop', 'militar

In [126]:
from sklearn.decomposition import NMF
n_features = 4000
n_components = 20
n_top_words = 20

# ignore terms that have a document frequency strictly higher than 95%, 
# ignore terms that have a document frequency strictly lower than 2
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(dataAP)

# alpha=0 means no regularization, l1_ratio=.5, the penalty is a combination of L1 and L2
nmf = NMF(n_components=n_components, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)
nmf_output = nmf.fit_transform(tfidf)

def show_topics(vectorizer=tfidf_vectorizer, lda_model=nmf, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=tfidf_vectorizer, lda_model=nmf, n_words=20)        


# Air Pollution Topics

In [127]:
topic_keywords

[array(['wors', 'suggest', 'think', 'impact', 'studi', 'health', 'pollut',
        'guardian', 'previous', 'environ', 'cell', 'bodi', 'affect',
        'dirti', 'googlenew', 'suggestsâ', 'hava', 'news', 'sadiqkhan',
        'review'], dtype='<U15'),
 array(['pollut', 'nois', 'water', 'control', 'today', 'area', 'high',
        'fight', 'toxic', 'countri', 'check', 'monitor', 'univers',
        'christchurch', 'level', 'affect', 'damag', 'outdoor', 'plus',
        'sourc'], dtype='<U15'),
 array(['airpollut', 'airqual', 'cleanaireu', 'health', 'climatechang',
        'cleanair', 'environ', 'today', 'delhipollut', 'delhirain',
        'research', 'clean', 'purpleair', 'delhiairqu', 'read', 'affect',
        'awar', 'tehran', 'forum', 'place'], dtype='<U15'),
 array(['âµgmâ³', 'cegli', 'messapica', 'martina', 'daili', 'alert',
        'averag', 'limit', 'campi', 'httpsna', 'costa', 'minzoni',
        'torchiarolodon', 'pollut', 'pleas', 'play', 'platform', 'plastic',
        'pleasant', '

In [121]:
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

In [122]:
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,wors,suggest,think,impact,studi,health,pollut,guardian,previous,environ,cell,bodi,affect,dirti,googlenew,suggestsâ,hava,news,sadiqkhan,review
Topic 1,pollut,nois,water,control,today,area,high,fight,toxic,countri,check,monitor,univers,christchurch,level,affect,damag,outdoor,plus,sourc
Topic 2,airpollut,airqual,cleanaireu,health,climatechang,cleanair,environ,today,delhipollut,delhirain,research,clean,purpleair,delhiairqu,read,affect,awar,tehran,forum,place
Topic 3,âµgmâ³,cegli,messapica,martina,daili,alert,averag,limit,campi,httpsna,costa,minzoni,torchiarolodon,pollut,pleas,play,platform,plastic,pleasant,plantat
Topic 4,link,problem,heart,previous,health,shortterm,diseas,exposur,number,cancer,stroke,condit,failur,studi,grow,lung,list,infect,research,tract
Topic 5,climatestrik,breathelif,airpollut,humanright,auster,cleanair,kill,chennai,climatecrisi,india,immedi,leader,kid,demand,action,climat,plastic,pleasant,pleas,playground
Topic 6,bori,johnson,fail,trust,chang,climat,dont,year,greenjennyjon,watch,report,hold,negat,pollut,buri,write,damn,real,huffpost,mayor
Topic 7,korea,shut,south,quarter,winter,plant,coal,coalfir,generat,tackl,reduc,pollut,power,month,suspend,southkorea,effort,curb,donald,electr
Topic 8,day,level,visit,higher,hospit,pollut,scientist,chelsea,admiss,scienc,cdnpoli,surpris,viral,news,high,lower,increas,trend,time,number
Topic 9,reduc,need,citi,climat,public,transport,environ,help,live,clean,issu,pollut,chang,india,like,traffic,level,action,cleanaireu,qualiti
